# Linear model

Nesse notebook, será desenvolvida uma análise preditiva com modelos lineares de regressão logística com regularização do tipo Lasso, Ridge e ElasticNet.

In [1]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path

# sklearn
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

In [2]:
%matplotlib inline

sns.set(font_scale=0.9)

# seed padrão
seed = 101

**Relembrando...**

Lasso(alpha = 0.1, normalize=True)
Ridge(alpha=0.1, normalize=True)

-----

classification_report(y_test, y_pred)  # o mesmo para confusion_matrix

-----
y_pred_proba = logreg.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_pred_proba)

false_pos_rate, true_pos_rate, thresholds = roc_curve(y_test, y_pred_proba)

-----

**Passo a Passo:**

1. Importar dados
2. Normalizar var. numericas
3. One Hot Encoding (var. categoricas)
4. Estimar Lasso (com GridSearch)
5. Verificar importância de Features
6. Estimar Ridge e Elastic Net
7. Comparar modelos e identificar o melhor nos critérios: precision, recall, F1, curva ROC, tempo de treino, tempo de avaliação (teste)
8. Montar pipeline com o melhor modelo e salvar melhor modelo em um pickle.

## Importando dataset

In [3]:
datapath = Path('../data')

with open(f'{datapath}/data.pickle', 'rb') as file:
    data = pickle.load(file)

In [4]:
data.head(2)

,status_check_acc,duration,cred_history,purpose,cred_amount,savings,employment_since,installment,other_deb,resid_since,...,other_installment,housing,n_creds,job,n_liable_people,tel,foreign,cred_score,gender,social_status
0,0 DM,6,critical,radio/television,1169.0,None,>= 7 years,4,None,4,...,none,own,2,skilled,1,1,1,0,M,single
1,<200 DM,48,all paid,radio/television,5951.0,<100 DM,< 4 years,2,None,2,...,none,own,1,skilled,1,0,1,1,F,divorced/separated


## Separando variáveis preditoras e dependente (X e y)

In [5]:
X = data.drop('cred_score', axis=1).copy()
y = data['cred_score'].copy()

## Normalizando variáveis numéricas contínuas

As variáveis numéricas, porém discretas, serão tratadas como categóricas. No entanto, não serão sujeitas ao processo de `OneHotEncoding`.

In [6]:
continuous_vars = [col for col in X.columns if X[col].dtype == 'float']
print(continuous_vars)

['cred_amount']


In [7]:
for var in continuous_vars:
    mean = np.mean(X[var])
    std = np.std(X[var])
    
    values = X[var].values
    
    standard_var = (values - mean)/std  # padronizando
    
    X[var] = standard_var

## OneHotEncoding

Nesse passo, vamos converter todas as variáveis categóricas em dummies.

In [8]:
categorical_vars = [col for col in X.columns if X[col].dtype not in ['int', 'float']]
numerical_vars = [col for col in X.columns if col not in categorical_vars]

print('categorical_vars: ',categorical_vars)
print('\nnumerical_vars: ',numerical_vars)

categorical_vars:  ['status_check_acc', 'cred_history', 'purpose', 'savings', 'employment_since', 'other_deb', 'property', 'other_installment', 'housing', 'job', 'tel', 'foreign', 'gender', 'social_status']

numerical_vars:  ['duration', 'cred_amount', 'installment', 'resid_since', 'age', 'n_creds', 'n_liable_people']


In [9]:
# criando dummies
X_dummy = pd.get_dummies(X[categorical_vars], drop_first=True)

In [10]:
# agregando ao conjunto de features X
X = pd.concat([X[numerical_vars], X_dummy], axis=1)

## Train Test Split

Nesse passo, vamos separar um conjunto de treino e um de teste (_holdout_) para posterior avaliação fora da amostra. Como a amostra contém apenas 1000 registros, reservaremos 20% para compor o conjunto de teste. Será selecionado o modelo com o melhor *cross_validation_score* dentro da amostra de treino.

**OBS:** é importante que seja definido o parâmetro `stratify` para que sejam mantidas as mesmas proporções de clientes com score bom e ruim observadas na amostra.

In [11]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, stratify=y, test_size=0.2, random_state=seed)

## Estimando modelo Lasso

Com esse modelo, esperamos ter uma primeira medida de _feature importance_ e uma primeira avaliação de performance para modelos lineares.

In [12]:
# grid de parâmetros para o Lasso
lasso_param_grid = {
    'alpha':np.linspace(0.01, 1, 10)
}

In [13]:
lasso = Lasso()
lasso_cv = GridSearchCV(lasso, lasso_param_grid, cv=3)  # 3 fold devido ao número de registros
lasso_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=Lasso(),
             param_grid={'alpha': array([0.01, 0.12, 0.23, 0.34, 0.45, 0.56, 0.67, 0.78, 0.89, 1.  ])})

In [14]:
print('Lasso best params:', lasso_cv.best_params_)
print('Lasso best score (R2):', lasso_cv.best_score_)

Lasso best params: {'alpha': 0.01}
Lasso best score (R2): 0.14133132821853375


## Estimando modelo Ridge

In [15]:
# grid de parâmetros para o Lasso
ridge_param_grid = {
    'alpha':np.linspace(0.01, 1, 10)
}

In [16]:
ridge = Ridge()
ridge_cv = GridSearchCV(ridge, ridge_param_grid, cv=3)
ridge_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=Ridge(),
             param_grid={'alpha': array([0.01, 0.12, 0.23, 0.34, 0.45, 0.56, 0.67, 0.78, 0.89, 1.  ])})

In [17]:
print('Ridge best params:', lasso_cv.best_params_)
print('Ridge best score (R2):', lasso_cv.best_score_)

Ridge best params: {'alpha': 0.01}
Ridge best score (R2): 0.14133132821853375


In [18]:
l = LogisticRegression()

In [19]:
l.fit(X_train, y_train)

/home/franklin/anaconda3/envs/cases/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [23]:
cross_val_score(Lasso(), X_train, y_train, cv=3)

array([-0.00597102, -0.00070344, -0.00080379])